## 変分アルゴリズム

このコースでは、変分アルゴリズムの詳細と、量子力学の変分定理に基づく近い将来のハイブリッド量子-古典アルゴリズムについて説明します。これらのアルゴリズムは、今日のフォールトトレラントでない量子コンピューターによって提供されるユーティリティを活用できるため、[量子優位性](gloss:advantage)を達成するための理想的な候補になります。

このコースでは次のことを学びます。

- 変分アルゴリズム設計ワークフローの各ステップ
- 各ステップに関連するトレードオフ
- [Qiskit Runtimeプリミティブ](https://qiskit.org/documentation/partners/qiskit_ibm_runtime/primitives.html)を使用して速度と精度を最適化する方法

このコースは、研究者や開発者が量子コンピューターの有用性を探求するための出発点となることを意図していますが、量子コンピューティング全般の理論的・基礎的な知識は[量子情報と計算の基礎](https://qiskit.org/learn/course/basics-quantum-information/)( [YouTubeシリーズ](https://www.youtube.com/playlist?list=PLOFEBzvs-VvqKKMXX4vbi4EB1uaErFMSO)としても視聴可能）を自由に探検してみてください。

## 簡素化されたハイブリッドワークフロー

![Variational Flow](images/variational_workflow.png)

Variational algorithms include several modular components that can be combined and optimized based on algorithm, software, and hardware advancements. This includes a *cost function* that describes a specific problem with a set of parameters, an *ansatz* to express the search space with these parameters, and an *optimizer* to iteratively explore the search space. During each iteration, the optimizer evaluates the cost function with the current parameters and selects the next iteration's parameters until it [converges](gloss:converge) on an optimal solution. The hybrid nature of this family of algorithms comes from the fact that the cost functions are evaluated using quantum resources and optimized through classical ones.

1. **Initialize problem**: Variational algorithms start by initializing the quantum computer in a *default state* $|0\rangle$, then transforming it to some desired (non-parametrized) state $|\rho\rangle$, which we will call *reference state*.

    This transformation is represented by the application of a unitary reference operator $U_R$ on the default state, such that $U_R|0\rangle = |\rho\rangle$.

2. **Prepare ansatz**: To begin iteratively optimizing from the default state $|0\rangle$ to the target state $|\psi(\vec\theta)\rangle$, we must define a *variational form* $U_V(\vec\theta)$ to represent a collection of parametrized states for our variational algorithm to explore.

    We refer to any particular combination of reference state and variational form as an ansatz, such that: $U_A(\vec\theta) := U_V(\vec\theta) U_R$. Ansatze will ultimately take the form of parametrized quantum circuits capable of taking the default state $|0\rangle$ to the target state $|\psi(\vec\theta)\rangle$.

    全体として次のようになります。

    $$
   \begin{aligned}
   |0\rangle \xrightarrow{U_R} U_R|0\rangle

   &amp; = |\rho\rangle \xrightarrow{U_V(\vec{\theta})} U_A(\vec{\theta})|0\rangle \[1mm]

   &amp; = U_V(\vec{\theta})U_R|0\rangle \[1mm]

   &amp; = U_V(\vec{\theta})|\rho\rangle \[1mm]

   &amp; = |\psi(\vec{\theta})\rangle \[1mm]

   \end{aligned}
   $$

3. **Evaluate cost function**: We can encode our problem into a *cost function* $C(\vec\theta)$ as a linear combination of Pauli operators, run on a quantum system. While this can be information about a physical system, such as energy or spin, we can also encode non-physical problems as well. We can leverage Qiskit Runtime primitives to address noise with error suppression and mitigation while evaluating our cost function.

4. **Optimize parameters**: Evaluations are taken to a classical computer, where a classical optimizer analyzes them and chooses the next set of values for the variational parameters. If we have a pre-existing optimal solution, we can set it as an *initial point* $\vec\theta_0$ to *bootstrap* our optimization. Using this *initial state* $|\psi(\vec\theta_0)\rangle$ could help our optimizer find a valid solution faster.

5. **Adjust ansatz parameters with results, and re-run**: the  entire process is repeated until the classical optimizer's finalization criteria are met, and an optimal set of parameter values $\vec\theta^*$ is returned. The proposed solution state for our problem will then be $|\psi(\vec\theta^*)\rangle = U_A(\vec\theta^*)|0\rangle$.

## 変分定理

A common goal of variational algorithms is to find the quantum state with the lowest or highest eigenvalue of a certain observable. A key insight we'll use is the *variational theorem* of quantum mechanics. Before going into its full statement, let us explore some of the mathematical intuition behind it.

### エネルギーと基底状態の数学的直感

In quantum mechanics, energy comes in the form of a quantum observable usually referred to as the *Hamiltonian*, which we'll denote by $\hat{\mathcal{H}}$. Let us consider its [spectral decomposition](gloss:decomposition):

$$
\hat{\mathcal{H}} = \sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|
$$

where $N$ is the dimensionality of the space of states, $\lambda_{k}$ is the $k$-th eigenvalue or, physically, the $k$-th energy level, and $|\phi_k\rangle$ is the corresponding [eigenstate](gloss:eigenstate): $\hat{\mathcal{H}}|\phi_k\rangle = \lambda_k |\phi_k\rangle$, the expected energy of a system in the (normalized) state $|\psi\rangle$ will be:

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

&amp; = \langle \psi |\bigg(\sum_{k=0}^{N-1} \lambda_k |\phi_k\rangle \langle \phi_k|\bigg) | \psi \rangle \[1mm]

&amp; = \sum_{k=0}^{N-1} \lambda_k \langle \psi |\phi_k\rangle \langle \phi_k| \psi \rangle \[1mm]

&amp; = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \[1mm]

\end{aligned}
$$

$\lambda_0\leq \lambda_k, \forall k$ を考慮すると、次のようになります。

$$
\begin{aligned}
\langle \psi | \hat{\mathcal{H}} | \psi \rangle

&amp; = \sum_{k=0}^{N-1} \lambda_k |\langle \psi |\phi_k\rangle|^2 \[1mm]

&amp; \geq  \sum_{k=0}^{N-1} \lambda_0 |\langle \psi |\phi_k\rangle|^2 \[1mm]

&amp; = \lambda_0 \sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle|^2 \[1mm]

&amp; = \lambda_0 \[1mm]

\end{aligned}
$$

Since `{latex} \{ |\phi_k\rangle \}_{k=0}^{N-1}` is an orthonormal basis, the probability of measuring $|\phi_{k} \rangle$ is $p_k = |\langle \psi |\phi_{k} \rangle |^2$, and the sum of all probabilities is such that $\sum_{k=0}^{N-1} |\langle \psi |\phi_k\rangle|^2 = \sum_{k=0}^{N-1}p_k = 1$. In short, the expected energy of any system is higher than the lowest energy or ground state energy:

$$
\langle \psi | \hat{\mathcal{H}} | \psi \rangle \geq \lambda_0.
$$

上記の引数は、有効な (正規化された) 量子状態 $|\psi\rangle$ に適用されるため、パラメーターベクトル $\vec\theta$ に依存するパラメーター化された状態 $|\psi(\vec\theta)\rangle$ を考慮することは完全に可能です。ここで「変分」の出番です。 $C(\vec\theta) := \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle$ で与えられるコスト関数を考えるとそれを最小化したい場合、最小値は常に次を満たします。

$$
\min_{\vec\theta} C(\vec\theta) = 
\min_{\vec\theta} \langle \psi(\vec\theta)|\hat{\mathcal{H}}|\psi(\vec\theta)\rangle \geq \lambda_0.
$$

The minimum value of $C(\vec\theta)$ will be the closest that one can get to $\lambda_0$ using the parametrized states $|\psi(\vec\theta)\rangle$, and equality will only be reached if there exists a parameter vector $\vec\theta^*$ such that $|\psi(\vec\theta^*)\rangle = |\phi_0\rangle$

### 量子力学の変分定理

If the (normalized) state $|\psi\rangle$ of a quantum system depends on a parameter vector $\vec\theta$, then the optimal approximation of the ground state (i.e. the eigenstate $|\phi_0\rangle$ with the minimum eigenvalue $\lambda_0$) is the one that minimizes the [expectation value](gloss:expectation) of the Hamiltonian $\hat{\mathcal{H}}$:

$$
\langle \hat{\mathcal{H}} \rangle(\vec\theta) := 
\langle \psi(\vec\theta) |\hat{\mathcal{H}}| \psi(\vec\theta) \rangle \geq 
\lambda_0
$$

変分定理がエネルギー最小値の観点から述べられている理由は、多くの数学的仮定が含まれているためです。

- For physical reasons, a finite lower bound to the energy $E \geq \lambda_0 &gt; -\infty$ needs to exist, even for $N\rightarrow\infty$.
- 通常上限は存在しません。

However, mathematically speaking, there is nothing special about the Hamiltonian $\hat{\mathcal{H}}$ beyond these assumptions, so the theorem can be generalized to other quantum observables and their eigenstates provided they follow the same constraints. Also, note that if finite upper bounds exist, the same mathematical arguments could be made for maximizing eigenvalues by swapping lower bounds for upper bounds.

このレッスンでは、変分アルゴリズムの概要を学習しました。以降のレッスンでは、各ステップとそれに関連するトレードオフについて詳しく説明します。